#### first step:  given a trained model.

In [1]:
# tensorflow

import tensorflow as tf
import os
from tensorflow.keras.datasets import cifar10
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
x_test = x_test/255.0
x_train = x_train/255.0
model = tf.keras.models.load_model('../checkpoint/')
optimizer = tf.keras.optimizers.SGD(0.001, momentum=0.9)
model.compile(
    optimizer,
    tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
model.evaluate(x_test,y_test,batch_size=256)

2022-10-07 13:03:02.477579: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:03:08.547316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b5:00.0, compute capability: 8.6
2022-10-07 13:03:40.804820: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


 3/40 [=>............................] - ETA: 2s - loss: 0.4000 - acc: 0.9414 

2022-10-07 13:03:44.949166: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


40/40 [==============================] - 13s 78ms/step - loss: 0.4139 - acc: 0.9372


[0.41388535499572754, 0.9372000098228455]

#### second step: calculate the similarity between layers and complete the modularization.

In [ ]:
import os
os.system('python analysis.py --experiment_dir ../checkpoint/')
os.system('python cka_extract.py --base_dir ../checkpoint/')
os.system('python modularity.py --base_dir ../checkpoint/ --threshold 0.99')


#### third step: remove some redundant layers and observe the change in accuracy.      

In [2]:
import pandas as pd
data = pd.read_pickle('../checkpoint/0.99_modules.pkl')
print('redundant blocks: ',end=' ')
for d in data[2]:
    if d[0]!=d[1]:
        print(d,end=' ')

redundant blocks:  [5, 9] [22, 35] [38, 49] [53, 54] 

In [1]:
import torch
layers = torch.load('../checkpoint/resnet110_block.pt')
print('the index corresponding to each residual block \'add\' operation')
print(layers)

the index corresponding to each residual block 'add' operation
[9, 16, 23, 30, 37, 44, 51, 58, 65, 72, 79, 86, 93, 100, 107, 114, 121, 128, 136, 143, 150, 157, 164, 171, 178, 185, 192, 199, 206, 213, 220, 227, 234, 241, 248, 255, 263, 270, 277, 284, 291, 298, 305, 312, 319, 326, 333, 340, 347, 354, 361, 368, 375, 382]


In [1]:
# remove the layer 48 49
os.system('python property.py --experiment_dir ../checkpoint/ --layer_indexes 48 49 --opt removal')
# add the layer 48 49
os.system('python property.py --experiment_dir ../checkpoint/ --layer_indexes 48 49 --opt add')

2022-10-07 13:06:58.580625: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:07:04.645574: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b1:00.0, compute capability: 8.6


[48, 49]
------------- remove 0 layers


2022-10-07 13:07:31.468828: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204
2022-10-07 13:07:39.810902: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


0.9372
------------- remove 1 layers
0.9365
------------- remove 2 layers
0.9287
[0.9372, 0.9365, 0.9287]


2022-10-07 13:07:59.906710: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:08:05.885195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b1:00.0, compute capability: 8.6


[48, 49]
------------- add 0 layers


2022-10-07 13:08:33.273767: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204
2022-10-07 13:08:41.611544: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


0.9372
------------- add 1 layers
0.9371
------------- add 2 layers
0.932
[0.9372, 0.9371, 0.932]


0